![banniere.jpg](https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png)

# <font color="#1d479b">Contexte</font>

Pour atteindre l'objectif de **ville neutre en émissions de carbone en 2050**, la ville de **Seattle** s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, nous devons tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

<hr width="50%" align="center"/>

Dans cette première partie, nous allons réaliser une **courte analyse exploratoire** après avoir nettoyé les données si besoin. Le but sera de déterminer les variables pertinentes ou d'en créer de nouvelles *(feature engineering)*.

# <font color="#1d479b">Sommaire</font>

1. [Chargement et adaptation des données de relevés](#section_1)     
    1.1. [Comparaison des colonnes des datasets](#section_1_1)     
    1.2. [Décompactage des données de localisation de 2015](#section_1_2)     
    1.3. [Description et nettoyage des données](#section_1_3)     
2. [Analyse exploratoire & Feature Engineering](#section_2)      
    2.1. [Les types de bâtiments](#section_2_1)     
    2.2. [Les années de construction](#section_2_2)     
    2.3. [Les corrélations linéaires](#section_2_3)     
    2.4. [Analyse des variables à prédire](#section_2_4)
3. [Dernières étapes de nettoyage](#section_3)
4. [Projection des établissements sur la carte de Seattle](#section_4)

## <font color="#337da4" id="section_1">1. Chargement et adaptation des données de relevés</font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set_style("whitegrid")
sns.color_palette("crest", as_cmap=True)

# suppress future warnings de pandas 1.3.0  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Lecture du dossier data Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
data_2015 = pd.read_csv("../input/sea-building-energy-benchmarking/2015-building-energy-benchmarking.csv")
data_2015.head()

In [3]:
data_2016 = pd.read_csv("../input/sea-building-energy-benchmarking/2016-building-energy-benchmarking.csv")
data_2016.head()

Après avoir visualisé les premières lignes de ces 2 datasets, on remarque déjà que les colonnes ne sont pas identiques. Identifions les différences :

### <font color="#2cb7b0" id="section_1_1">1.1. Comparaison des colonnes des datasets</font>

In [4]:
def compare_colums(df1,df2):
    columns_1 = list(df1.columns) 
    columns_2 = list(df2.columns)
    same_columns=[]
    diff_columns_2=[]
    diff_columns_1=[]

    for col in columns_2:
        if col in columns_1:
            same_columns.append(col)
        else:
            diff_columns_2.append(col)
    for col in columns_1:
        if col not in columns_2:
            diff_columns_1.append(col)
    return diff_columns_1, diff_columns_2

In [5]:
diff_columns_2015, diff_columns_2016 = compare_colums(data_2015,data_2016)
diff_columns_2015

In [6]:
diff_columns_2016

Les **données de localisation** ont évolué entre 2015 et 2016. On retrouve en plus l'adresse, la ville et la localisation GPS a été segmentée en `Latitude` et `Longitude`. Certaines autres variables comme `GHGEmissions(MetricTonsCO2e)` ont changé de nom *(il faudra vérifier si l'ordre de grandeur des données à changer comparativement à* `TotalGHGEmissions` *de 2016)*.

### <font color="#2cb7b0" id="section_1_2">1.2. Décompactage des données de localisation de 2015</font>

In [7]:
data_2015['Location'][0]

On remarque que les données de localisation pour le jeu de données de 2015 sont "compactées" dans une sorte de double dictionnaire. Nous allons donc travailler cette variable pour extraire chacune des variables imbriquées :

In [8]:
import ast
data_2015['Location'] = [ast.literal_eval(str(item)) for index, item in data_2015.Location.iteritems()]
data_2015 = pd.concat([data_2015.drop(['Location'], axis=1), data_2015['Location'].apply(pd.Series)], axis=1)
data_2015['human_address'] = [ast.literal_eval(str(item)) for index, item in data_2015.human_address.iteritems()]
data_2015 = pd.concat([data_2015.drop(['human_address'], axis=1), data_2015['human_address'].apply(pd.Series)], axis=1)
data_2015.head()

Nous avons à présent les colonnes correspondant à celles de 2016 : `latitude`, `longitude`, `address`, `city`, `state` et `zip`. Renommons les de la même façon :

In [9]:
data_2015 = data_2015.rename(columns={"latitude":"Latitude", "longitude":"Longitude",
                                      "address":"Address", "city":"City", 
                                      "state":"State", "zip":"ZipCode"})

Puis regardons à nouveau les différences de colonnes entre les 2 dataframes :

In [10]:
diff_columns_2015, diff_columns_2016 = compare_colums(data_2015,data_2016)
diff_columns_2015

In [11]:
diff_columns_2016

In [12]:
print(f"Description de la variable TotalGHGEmissions 2016 : \n\n",data_2016['TotalGHGEmissions'].describe(),
     f"\n\nDescription de la variable GHGEmissions(MetricTonsCO2e) 2015 : \n\n", data_2015['GHGEmissions(MetricTonsCO2e)'].describe())

Les ordres de grandeur des 2 variables sont identiques entre 2015 et 2016. Nous allons donc simplement renommer les colonnes à l'identique. Nous supprimons également les colonnes de 2015 n'ayant pas d'équivalent en 2016:

In [13]:
data_2015 = data_2015.rename(columns={'GHGEmissions(MetricTonsCO2e)':'TotalGHGEmissions',
                                     'GHGEmissionsIntensity(kgCO2e/ft2)':'GHGEmissionsIntensity',
                                     'Comment':'Comments'})
data_2015.drop(['OtherFuelUse(kBtu)','2010 Census Tracts',
                'Seattle Police Department Micro Community Policing Plan Areas',
                'City Council Districts','SPD Beats', 'Zip Codes'], axis=1, inplace=True)

In [14]:
diff_columns_2015, diff_columns_2016 = compare_colums(data_2015,data_2016)
print(diff_columns_2015,diff_columns_2016)

**Les variables des 2 datasets étant à présent identiques**, nous allons pouvoir les regrouper en un unique jeu de données :

In [15]:
data = pd.concat([data_2015[data_2016.columns],data_2016], axis = 0).sort_values(["DataYear", "OSEBuildingID"])
data.shape

### <font color="#2cb7b0" id="section_1_3">1.3. Description et nettoyage des données</font>

Il est précisé dans le projet que **seuls les bâtiments non destinés à l'habitation seront étudiés**. Nous allons donc supprimer toutes les lignes correspondant à des habitations en nous basant sur la variable `BuildingType`

In [16]:
data['BuildingType'].unique()

In [17]:
data = data[~data['BuildingType'].str.contains("Multifamily")]
data['BuildingType'].unique()

In [18]:
print("Le jeu de données compte à présent {} lignes et {} colonnes.".format(data.shape[0],data.shape[1]))

Nous allons également regarder s'il existe des **doublons sur l'identifiant** `OSEBuildingID`. En effet, nos modélisations devront porter sur un bâtiment unique *(ce n'est pas une modélisation temporelle)*. Nous prendrons donc en valeur la moyenne des variables sur les 2 années :

In [19]:
mean_columns = ['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
                'PropertyGFAParking', 'PropertyGFABuilding(s)',
                'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
                'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
                'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
                'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
                'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)',
                'NaturalGas(kBtu)', 'TotalGHGEmissions', 'GHGEmissionsIntensity']
OSEBuilding_means = data[['OSEBuildingID']+mean_columns].groupby('OSEBuildingID').mean()
OSEBuilding_means.head()

In [20]:
duplicate_building = data.drop_duplicates(subset=['OSEBuildingID'], keep='last')
duplicate_building.drop(mean_columns, axis=1, inplace=True)
data = pd.merge(duplicate_building, OSEBuilding_means, how='left', on='OSEBuildingID')

Le jeu de données ne compte à présent plus de doublons sur la variable `OSEBuildingID`.

Regardons à présent les infos et descriptions du dataset :

In [21]:
data.info()

Dans la visualisation ci-dessus, certaines variables apparaissent déjà comme redondantes :
- `Electricity(kWh)` et `Electricity(kBtu)`,
- `NaturalGas(therms)` et `NaturalGas(kBtu)`
- Les suffixes **WN** : "Weather Normalized" - Ce sont les mesures normalisées avec les conditions climatiques. Dans le cadre de notre analyse, la météo ne rentrera pas en compte.

Nous allons donc commencer par supprimer ces variables :

In [22]:
def search_componant(df, suffix=None):
  componant = []
  for col in df.columns:
      if suffix in col: 
        componant.append(col)
  return componant

In [23]:
#Suppression des variables WN
data.drop(search_componant(data,'WN'), axis=1, inplace=True)

In [24]:
#Suppression des variables redondantes
redundant_features = ['NaturalGas(therms)','Electricity(kWh)']
data.drop(redundant_features, axis=1, inplace=True)

On remarque également des variables suffixées **GFA** : Elles représente la surface au sol *(Ground Floor Area)*. Nous les conservons donc pour la suite des analyses.

In [25]:
data.describe()

In [26]:
def not_nulls(df):
    ret = df.notnull().sum().to_frame(name='count').rename_axis('column').reset_index()
    return ret


def plot_not_null(df, titre='nombre de valeurs not null', figsize=None, orient='h'):
    """affiche un bar plot avec le nombre de valeurs not null"""

    _, ax = plt.subplots(figsize=figsize)

    if orient == 'h':
        ax1 = sns.barplot(ax=ax, y=df.columns,
                          x=df.notnull().sum(), orient='h')
        ax1.set(ylabel="colonne", xlabel='count not null')
        ax1.xaxis.grid()
    else:
        ax1 = sns.barplot(ax=ax, x=df.columns,
                          y=df.notnull().sum(), orient='v')
        plt.xticks(rotation=90)
        ax1.set(xlabel="colonne", ylabel='count not null')
        # ax1.yaxis.grid()

    ax1.set_title(titre)
    sns.despine()
    ax1.plot()
    return ax1

In [27]:
plot_not_null(data, titre='Nombre de valeurs remplis dans chaque colonne ',
              figsize=(25, 5), orient='v');

Dans le cadre de nos modélisations, les variables à prédire sont la **consommation d'énergie du bâtiment** (`SiteEnergyUse(kBtu)`) et ses **émissions de CO2** (`TotalGHGEmissions`). Certaines lignes comportent des manquants sur ces variables, nous allons donc les supprimer :

In [28]:
data = data[~((data['SiteEnergyUse(kBtu)'].isnull()) | (data['TotalGHGEmissions'].isnull()))]

La variable `Comments`, très peu renseignée également, peux être supprimée :

In [29]:
data.drop("Comments", axis=1, inplace=True)

La variable identifiant les outliers peut être interessante pour nos analyses, cependant, dans la documentation, nous ne savons pas rééllement à quoi correspondent ces outliers. Nous allons donc supprimer les lignes mentionnant ces outliers.

In [30]:
data = data[~data["Outlier"].isnull()==False]
data.drop('Outlier', axis=1, inplace=True)

Nous allons pour le moment conserver les autres variables en l'état. Une courte analyse exploratoire nous en apprendra plus sur les données à conserver.

## <font color="#337da4" id="section_2">2. Analyse exploratoire & Feature Engineering</font>

Dans un premier temps, nous allons regarder la répartition des divers types de bâtiments à étudier :

### <font color="#2cb7b0" id="section_2_1">2.1. Les types de bâtiments</font>

In [31]:
building_type = data.groupby(by='BuildingType')['OSEBuildingID'].nunique()

font_title = {'family': 'serif',
              'color':  '#1d479b',
              'weight': 'bold',
              
             }

fig, ax = plt.subplots(figsize=(8,8))
ax.pie(building_type.values, labels=building_type.index, 
       autopct='%1.1f%%', shadow=True, startangle=30,
       textprops=dict(color="black",size=12, weight="bold"))
ax.axis('equal')
ax.set_title("Répartition des types de bâtiments du Dataset", fontdict=font_title, fontsize=22)
plt.show()

La majeure partie des bâtiments sont typés **"NonResidential"**. Nous pouvons visualiser les diverses catégories représentées dans ce type de bâtiments :

In [32]:
data.loc[(data['BuildingType']=="NonResidential"),'PrimaryPropertyType'].value_counts()

Ici, on remarque que des catégories sont des **doublons avec un caractère d'échappement**. Nous allons corriger ce problème :

In [33]:
import re
regex = re.compile(r'[\n\r\t]')
data['PrimaryPropertyType'] = [regex.sub("", item) for index, item in data.PrimaryPropertyType.iteritems()]
data.loc[(data['BuildingType']=="NonResidential"),'PrimaryPropertyType'].value_counts()

Les bureaux de petite et moyenne taille représentent la plus grande part des bâtiments non résidentiels.

### <font color="#2cb7b0" id="section_2_2">2.2. Les années de construction</font>
Nous allons regarder les distribution des années de construction des bâtiments de Seattle :

In [34]:
fig = plt.figure(figsize=(12,8))
ax = sns.histplot(data=data, x='YearBuilt', bins=int((data.YearBuilt.max() - data.YearBuilt.min())/5))
ax.set_xlabel("Année de construction")
ax.set_ylabel("Nombre de bâtiments")
plt.title(f"Distribution des années de construction des bâtiments\n", fontdict=font_title)
plt.show()

Plus que l'année de construction, il serait intéressant de traiter l'**age des bâtiments** pour réduire la dispersion des données et lier l'année des relevés. Nous allons donc créer cette nouvelle variable et supprimer l'année de construction :

In [35]:
data['BuildingAge'] = data['DataYear'] - data['YearBuilt']
data.drop('YearBuilt', axis=1, inplace=True)

fig = plt.figure(figsize=(12,8))
ax = sns.histplot(data=data, x='BuildingAge', bins=int((data.BuildingAge.max() - data.BuildingAge.min())/5))
ax.set_xlabel("Age du bâtiment")
ax.set_ylabel("Nombre de bâtiments")
plt.title(f"Distribution de l'âge des bâtiments\n", fontdict=font_title)
plt.show()

### <font color="#2cb7b0" id="section_2_3">2.3. Les corrélations linéaires</font>

In [36]:
corr = data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(corr, annot=True, fmt=".2f", annot_kws={'size':8}, 
                 mask=mask, center=0, cmap="coolwarm")
plt.title(f"Heatmap des corrélations linéaires\n", 
          fontdict=font_title)
plt.show()

Pour les variables à prédire `TotalGHGEmissions` et `SiteEnergyUse(kBtu)`, on remarque des corrélations linéaires quasi similaires avec les variables de relevés (les consommations) mais également avec le nombre de batiments ou d'étages ains que les surfaces au sol. 

On remarque sur ce Heatmap de fortes corrélations linéaires entre variables. Ces corrélations peuvent amener des problèmes de colinéarité dans nos futurs modèles. Isolons donc les **paires de variables avec des corrélations de Pearson supérieurs à 0.7** :

In [37]:
threshold = 0.7
corr_pairs = corr.unstack().sort_values(kind="quicksort")
strong_corr = (pd.DataFrame(corr_pairs[(abs(corr_pairs) > threshold)])
               .reset_index().rename(columns={0:'corr_coeff'}))
strong_corr = strong_corr[(strong_corr.index%2 == 0) & (strong_corr['level_0'] != strong_corr['level_1'])]
strong_corr.sort_values('corr_coeff', ascending=False)

On remarque que les variables suffixées GFA présentent de fortes corrélations avec plusieurs autres variables. Nous allons donc **créer de nouvelles variables** pour tenter de gommer ces corrélations linéaires :

In [38]:
def split_words(df, column = None):
  list_words = set()
  for word in df[column].str.split(','):
    if isinstance(word, float):
      continue
    list_words = set().union(word, list_words)
  return list(list_words)

list_use_type = split_words(data, 'ListOfAllPropertyUseTypes')
print("Nombre de type d'usages dans la base : {}".format(len(list_use_type)))

Réaliser un OneHotEncoder sur 117 types d'usage ne serait pas opportun. Nous allons donc créer une variable nous donnant le **nombre total d'usage du bâtiment**, puis supprimer la liste complète des usages :

In [39]:
data['TotalUseTypeNb'] = [str(word).count(",") + 1 for word in data['ListOfAllPropertyUseTypes'].str.split(',')]
data.drop('ListOfAllPropertyUseTypes', axis=1, inplace=True)

Nous allons à présent convertir les différentes surfaces (Buildings et Parking) en **pourcentage de la surface totale** et nous conserverons uniquement ces 2 variables en supprimant les variables `LargestPropertyUseTypeGFA`, `SecondLargestPropertyUseTypeGFA`, `ThirdLargestPropertyUseTypeGFA` :

In [40]:
gfa_features = search_componant(data, suffix='GFA')
data[['TotalUseTypeNb'] + gfa_features].head(10)

In [41]:
#On calcule les ratios
data['GFABuildRate'] = (round((data['PropertyGFABuilding(s)'].fillna(0)
                                  /data['PropertyGFATotal'].fillna(0)),5))
data['GFAParkRate'] = (round((data['PropertyGFAParking'].fillna(0)
                                 /data['PropertyGFATotal'].fillna(0)),5))

#On supprime les variables inutiles
data.drop(['LargestPropertyUseTypeGFA', 
           'SecondLargestPropertyUseTypeGFA',
           'SecondLargestPropertyUseType',
           'ThirdLargestPropertyUseTypeGFA',
           'ThirdLargestPropertyUseType',
           'PropertyGFAParking',
           'PropertyGFABuilding(s)'],
         axis=1, inplace=True)

#On complète les usages de la partie la plus large
data['LargestPropertyUseType'] = data['LargestPropertyUseType'].fillna("Unknown")
data['NumberofFloors'] = data['NumberofFloors'].fillna(1)

Nous pouvons également calculer la **surface moyenne par bâtiment et par étage** :

In [42]:
data['GFAPerBuilding'] = round((data['PropertyGFATotal'] / data['NumberofBuildings']),3)
data["NumberofFloors"][data["NumberofFloors"] == 0] = 1
data['GFAPerFloor'] = round((data['PropertyGFATotal'] / data['NumberofFloors']),3)

In [43]:
data.info()

Les données sont à présent bien complétées. Nous allons vérifier l'impact de ce feature engineering sur la matrice des corrélations linéaires :

In [44]:
corr = data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(corr, annot=True, fmt=".2f", annot_kws={'size':8}, 
                 mask=mask, center=0, cmap="coolwarm")
plt.title(f"Heatmap des corrélations linéaires\n", 
          fontdict=font_title)
plt.show()

In [45]:
threshold = 0.7
corr_pairs = corr.unstack().sort_values(kind="quicksort")
strong_corr = (pd.DataFrame(corr_pairs[(abs(corr_pairs) > threshold)])
               .reset_index().rename(columns={0:'corr_coeff'}))
strong_corr = strong_corr[(strong_corr.index%2 == 0) & (strong_corr['level_0'] != strong_corr['level_1'])]
strong_corr.sort_values('corr_coeff', ascending=False)

Vérification de **multicolinéarité avec le VIF** *(Variance Inflation Factor)* : $VIF = \frac{1}{1-R^2}$

In [46]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

strong_corr_features = list(set(list(strong_corr['level_0'].values) + list(strong_corr['level_1'].values)))
X = data[strong_corr_features].replace([np.inf, -np.inf], np.nan)
X = X.dropna()

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                   for i in range(len(X.columns))]
vif_data[vif_data['VIF'] > 5]

Des scores VIF supérieur à 5 indiquent généralement une forte multicolinéarité. Ces variables fortement corrélées risquent d'impacter nos modèles.       
Les features suffixées `EUI(kBtu/sf)`, sont des variables dont les valeurs sont ramenées à la surface par étage. Nous allons les supprimer car nous avons créer des variables pouvant permettre de ramener nos données à l'étage ou au building. Idem pour la variable `GHGEmissionsIntensity` 

In [47]:
Eui_features = search_componant(data, suffix='EUI(kBtu/sf)') + ['GHGEmissionsIntensity']
data.drop(Eui_features, axis=1, inplace=True)

### <font color="#2cb7b0" id="section_2_4">2.4. Analyse des variables à prédire</font>

Pour rappel, les 2 variables à prédire dans le cadre de notre mission sont :
- `TotalGHGEmissions`
- `SiteEnergyUse(kBtu)`

Nous allons donc réaliser quelques analyses exploratoires sur ces features :

In [48]:
import scipy.stats as stats

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))
left, width = 0, 1
bottom, height = 0, 1
right = left + width
top = bottom + height

sns.histplot(data=data, x="TotalGHGEmissions", kde=True, ax=axes[0], color="#9C3E2D", alpha=0.6)
axes[0].set_title("Données d'emission de CO2 globales", color='#2cb7b0')

#Test de Kolmogorov-Smirnov
kstest = stats.kstest(data['TotalGHGEmissions'].notnull(),'norm')
axes[0].text(right, top, 'Test Kolmogorov-Smirnov \n Pvalue: {:.2} \n Stat: {:.2}'.format(kstest.pvalue, kstest.statistic),
            horizontalalignment='right',
            verticalalignment='top',
            style='italic', transform=axes[0].transAxes, fontsize = 12,
            bbox={'facecolor':'#00afe6', 'alpha':0.5, 'pad':0})

sns.histplot(data=data[(data['TotalGHGEmissions']< 1000)], x="TotalGHGEmissions", kde=True, ax=axes[1], color="#9C3E2D", alpha=0.6)
axes[1].set_title("Données d'emission de CO2 zoomées", color='#2cb7b0')

plt.suptitle("Distribution des emissions de CO2 relevées (2015-2016)", 
              fontdict= font_title, fontsize=22)
plt.show()

In [49]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))

sns.histplot(data=data, x="SiteEnergyUse(kBtu)", kde=True, ax=axes[0], color="#6D9C0E", alpha=0.6)
axes[0].set_title("Données de consommation d'énergie globales", color='#2cb7b0')

#Test de Kolmogorov-Smirnov
kstest = stats.kstest(data['SiteEnergyUse(kBtu)'].notnull(),'norm')
axes[0].text(right, top, 'Test Kolmogorov-Smirnov \n Pvalue: {:.2} \n Stat: {:.2}'.format(kstest.pvalue, kstest.statistic),
            horizontalalignment='right',
            verticalalignment='top',
            style='italic', transform=axes[0].transAxes, fontsize = 12,
            bbox={'facecolor':'#00afe6', 'alpha':0.5, 'pad':0})

sns.histplot(data=data[(data['SiteEnergyUse(kBtu)']< 0.3*10**8)], x="SiteEnergyUse(kBtu)", kde=True, ax=axes[1], color="#6D9C0E", alpha=0.6)
axes[1].set_title("Données de consommation d'énergie zoomées", color='#2cb7b0')

plt.suptitle("Distribution des consommation d'énergie relevées (2015-2016)", 
              fontdict= font_title,fontsize=22)
plt.show()

En se basant sur les projections obtenus et les résultats des tests de Kolmogorov-Smirnov *(Pvalue < au niveau de test de 5%)* on rejette donc l'hypothèse de normalité des distributions de ces variables. 

Projettons à présent les scatterplots des distributions de ces 2 variables entre elles :

In [50]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))
sns.scatterplot(data=data, x="TotalGHGEmissions", y="SiteEnergyUse(kBtu)", ax=axes[0])
axes[0].set_title("Données globales", color='#2cb7b0')
sns.scatterplot(data=data[(data['TotalGHGEmissions'] < 5000)], x="TotalGHGEmissions", y="SiteEnergyUse(kBtu)", ax=axes[1])
axes[1].set_title("Données zoomées", color='#2cb7b0')
plt.suptitle("Répartition des données de consommation d'énergie vs emissions de CO2", fontdict=font_title, fontsize=22)
plt.show()

On remarque ici que la répartition des données d'emission de CO2 en fonction de la consommation d'énergie ne suivent pas uniquement 1 seule droite de régression linéaire si l'on zoom sur les données les plus représentées.

Regardons à présent si les **coordonnées géographiques** ont un impact sur les rejets et consommations. Pour cela, afin d'éviter les corrélations fortes entre Latitude et Longitude, nous allons calculer la **distance Harversine entre chaque point de coordonnées et le centre de Seattle** :

In [51]:
from math import radians, cos, sin, asin, sqrt

#Coordonnées du centre de Seattle
seattle_lat = 47.6062
seattle_lon = -122.3321

def haversine_distance(lat1, lng1, lat2, lng2, degrees=True):
    r = 3956 # rayon de la Terre en miles
    
    if degrees:
        lat1, lng1, lat2, lng2 = map(radians, [lat1, lng1, lat2, lng2])
    
    # Formule Haversine
    dlng = lng2 - lng1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlng/2)**2
    d = 2 * r * asin(sqrt(a))  

    return d

In [52]:
#Calcul des distance au centre de Seattle pour chaque point
data['harvesine_distance'] = [haversine_distance(seattle_lat, seattle_lon, x, y) 
                              for x, y in zip(data.Latitude.astype(float), data.Longitude.astype(float))]

In [53]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))
sns.scatterplot(data=data, y="TotalGHGEmissions", x="harvesine_distance", color="#9C3E2D", ax=axes[0])
axes[0].set_title("Données globales", color='#2cb7b0')
sns.histplot(data=data[(data['TotalGHGEmissions'] < 2500)], y="TotalGHGEmissions", 
                x="harvesine_distance", color="#9C3E2D", ax=axes[1])
axes[1].set_title("Données zoomées", color='#2cb7b0')
plt.suptitle("Répartition des données d'emissions de CO2 en fonction des coordonnées géographiques", 
             fontdict=font_title, fontsize=22)
plt.show()

In [54]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))
sns.scatterplot(data=data, y="SiteEnergyUse(kBtu)", x="harvesine_distance", color="#6D9C0E", ax=axes[0])
axes[0].set_title("Données globales", color='#2cb7b0')
sns.histplot(data=data[(data['SiteEnergyUse(kBtu)'] < 2*10**8)], y="SiteEnergyUse(kBtu)", 
                x="harvesine_distance", color="#6D9C0E", ax=axes[1])
axes[1].set_title("Données zoomées", color='#2cb7b0')
plt.suptitle("Répartition des données d'emissions de CO2 en fonction des coordonnées géographiques", 
             fontdict=font_title, fontsize=22)
plt.show()

En regardant ces projections, il semble que **les coordonnées géographiques *(donc les adresses des bâtiments)* puissent avoir un impact sur les consommations d'énergie et rejets de CO2**.

D'autre part, la latitude et la longitude étant 2 variables fortement corrélées dans notre jeu de données, **nous allons supprimer ces 2 colonnes pour conserver uniquement ce point de coordonnée unique Harvesine** *(en fin de Notebook)*.

Nous allons à présent regarder la répartition de ces 2 variables en fonction du type de bâtiment.

In [55]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(20,8))
sns.barplot(x='BuildingType',y='TotalGHGEmissions',data=data, ax=axes[0])
sns.barplot(x='BuildingType',y='SiteEnergyUse(kBtu)',data=data, ax=axes[1])
plt.suptitle("Répartition de la consommation d'énergie et emissions de CO2 en fonction du type de bâtiment", 
             fontdict=font_title, fontsize=18)
plt.show()

Sur ces diagrammes en barre, les campus se démarquent largement en terme de consommation et de rejets de CO2. Regardons à présent si l'âge des bâtiments a un impact sur les émissions de CO2 :

In [56]:
data["TotalGHGEmissions"].value_counts()

In [57]:
data['BuildingAge']

In [58]:
def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning. 
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0 
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    

    bins = []
    for low in range(lower_bound, 
                     lower_bound + quantity*width + 1, width):
        bins.append((low, low+width))
    return bins

In [59]:
bins = create_bins(lower_bound=0,
                   width=10,
                   quantity=11)
bins = pd.IntervalIndex.from_tuples(bins)

In [60]:
data["Building_age_bins"] = pd.cut(data['BuildingAge'], bins=bins)

In [61]:
sns.catplot(
    data=data, kind="bar",
    x="Building_age_bins", y="TotalGHGEmissions",
    ci=None, color="#9C3E2D", alpha=.6,
    height=7, aspect=2
)
plt.title("Influence de l'âge des bâtiments sur les émissions de CO2", fontdict=font_title, fontsize=22)
plt.show()

In [62]:
sns.catplot(
    data=data, kind="bar",
    x="Building_age_bins", y="SiteEnergyUse(kBtu)",
    ci=None, color="#6D9C0E", alpha=.6,
    height=7, aspect=2
)
plt.title("Influence de l'âge des bâtiments sur les consommations d'énergie", fontdict=font_title)
plt.show()

Les bâtiments de moins de 30 ans semblent avoir des consommations d'énergie et rejets de CO2 plus important que les buildings anciens, alors même que la variable `BuildingAge` n'est pas fortement corrélée à d'autres features *(comme la taille des bâtiments par exemple)*.

## <font color="#337da4" id="section_3">3. Dernières étapes de nettoyage</font>

Nous allons éliminer certaines variables qui ne seront pas utiles pour nos modélisations et vérifier les données incomplètes identifiées dans le jeu de données initial.

In [63]:
data.info()

Vérifions la variable `ComplianceStatus` qui représente la conformité des données relevées :

In [64]:
data['ComplianceStatus'].unique()

In [65]:
print("Nombre de ligne identifiées comme non conforme : {}.".format(data[data['ComplianceStatus'] != "Compliant"].shape[0]))

In [66]:
data = data[data['ComplianceStatus'] == "Compliant"]

Nous allons ensuite supprimer les variables `DefaultData`, `ComplianceStatus`, `TaxParcelIdentificationNumber`, `CouncilDistrictCode`, `City` 

In [67]:
data = data.drop(['DefaultData','ComplianceStatus', 'City',
                  'TaxParcelIdentificationNumber','CouncilDistrictCode'], axis=1)

## <font color="#337da4" id="section_4">4. Projection des établissements sur la carte de Seattle</font>

In [68]:
import folium
import folium.plugins

seattle_map = folium.Map(location=[seattle_lat, seattle_lon], zoom_start=11)

#Clusters
marker_cluster = folium.plugins.MarkerCluster().add_to(seattle_map)
for lat, lng, in zip(data.Latitude, data.Longitude):
    folium.Marker(location=[lat, lng]).add_to(marker_cluster)

seattle_map

Et pour finir, nous supprimons les variables `Latitude` et `Longitude` puis **nous exportons le fichier cleané pour les modélisations qui seront effectuées dans un second Notebook** 

In [69]:
data.info()

In [70]:
data.drop(['Latitude','Longitude'], axis=1,inplace=True)


Comme nous l'avons remarqué , les **Campus** sont très consommateurs en energie. Nous allons vérifier la distribution des consommations en fonction des surfaces totales au sol par catégorie de bâtiment. D'éventuels valeurs hors-normes pourraient poser des problèmes pour les modélisations :

In [71]:
fig = plt.figure(figsize=(12,8))
sns.scatterplot(data = data, x='PropertyGFATotal', y='SiteEnergyUse(kBtu)', hue='BuildingType')
plt.title(f"Consommations d'énergie par surface totale au sol et par type de bâtiment\n", 
          fontdict=font_title, fontsize=16)
plt.show()

Un bâtiment de type campus est très supérieur aux autres données. Il ne s'agit sans doute pas d'une valeur aberrante mais d'une valeur atypique qui est très isolée. Nous allons ici la supprimer de nos données sources.

In [72]:
data[data['SiteEnergyUse(kBtu)']>8*10**8]

In [73]:
data = data[data['SiteEnergyUse(kBtu)']<8*10**8]

Le but de notre programme est de supprimer les relevés couteux pour les années à venir. Nous allons donc exclure toutes les données de relevé de notre dataset.

In [74]:
data = data.drop(['SteamUse(kBtu)','Electricity(kBtu)',
                         'NaturalGas(kBtu)','Building_age_bins'], axis=1)

In [75]:
data.info()

In [76]:
corr = data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(corr, annot=True, fmt=".2f", annot_kws={'size':8}, 
                 mask=mask, center=0, cmap="coolwarm")
plt.title(f"Heatmap des corrélations linéaires\n", 
          fontdict=font_title)
plt.show()

In [77]:

data.set_index("OSEBuildingID").to_csv("SEA-cleaned.csv")